In [100]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import RepeatVector
from keras.layers.wrappers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers import Reshape
from embeddings import Embeddings
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing import sequence
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.utils import to_categorical
from keras.layers.pooling import MaxPool1D

In [2]:
embedding = Embeddings(100, 3, 1, 4)

In [3]:
word2vec_model = embedding.get_model()
vocab = embedding.get_vocabulary()

In [4]:
word2index = vocab[0]

In [5]:
contexts = ["""Tesla gained experience in telephony and electrical engineering before emigrating to the United States in 1884 to work for Thomas Edison in New York City.
He soon struck out on his own with financial backers, setting up laboratories and companies to develop a range of electrical devices.
His patented AC induction motor and transformer were licensed by George Westinghouse, who also hired Tesla for a short time as a consultant.
His work in the formative years of electric power development was involved in a corporate alternating current/direct current "War of Currents" as well as various patent battles.
""","""Tesla went on to pursue his ideas of wireless lighting and electricity distribution in his high-voltage, high-frequency power experiments in New York and Colorado Springs, and made early (1893) pronouncements on the possibility of wireless communication with his devices. 
          He tried to put these ideas to practical use in an ill-fated attempt at intercontinental wireless transmission, his unfinished Wardenclyffe Tower project.
          In his lab he also conducted a range of experiments with mechanical oscillators/generators, electrical discharge tubes, and early X-ray imaging.
          He also built a wireless controlled boat, one of the first ever exhibited."""]

In [6]:
max_len = max([len(word_tokenize(context)) for context in contexts])
"""tokenized_contexts = [word_tokenize(context) for context in contexts]
indexed_contexts = [[word2index[word] for word in context if word in word2index] for context in tokenized_contexts]"""

'tokenized_contexts = [word_tokenize(context) for context in contexts]\nindexed_contexts = [[word2index[word] for word in context if word in word2index] for context in tokenized_contexts]'

In [7]:
# padded_contexts = sequence.pad_sequences(indexed_contexts)

In [8]:
target = np.zeros(max_len)

In [9]:
question = "In what year did Nikola Tesla emigrate to the United States?"

In [10]:
from itertools import product

In [11]:
context_question = list(product(contexts, [question]))

In [12]:
# tokenized_sent = [word_tokenize(sent) for sent in sent_tokenize(context)]

In [13]:
def get_average_sentence_vector(sentence):
    avg_sent_vec = np.zeros((100))
    count = 0
    for word in sentence:
        if word in word2vec_model.wv.vocab:
            avg_sent_vec += word2vec_model[word]
            count+=1
    avg_sent_vec = np.divide(avg_sent_vec, count)
    return avg_sent_vec

In [14]:
avg_sentences = []
for context, question in context_question:
    avg_sentences.append(get_average_sentence_vector(word_tokenize(context)))

In [15]:
avg_sentences = np.array(avg_sentences)

In [16]:
avg_question_vec = get_average_sentence_vector(word_tokenize(question))

In [17]:
x = np.array(list(product(avg_sentences, [avg_question_vec])))

In [18]:
x = x.reshape((2, 200))

In [28]:
sent_1_answer_start = target
sent_1_answer_start[13] = 1
sent_2_answer_start = target
sent_1_answer_end = target
sent_1_answer_end[13] = 1
sent_2_answer_end = target
y_start = np.array([sent_1_answer_start, sent_2_answer_start])
y_end = np.array([sent_1_answer_end, sent_2_answer_end])

In [29]:
x = x.reshape(2,200)
# y_start = y_start.reshape(2, 2106)

In [32]:
model = Sequential()
model.add(Dense(100, input_shape=(200,)))
model.add(RepeatVector(1))
model.add(Bidirectional(LSTM(100)))
# model.add(Reshape((2, 1, 100)))
model.add(Dense(y.shape[3], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()
model.fit(x, y_start, epochs=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 100)               20100     
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 1, 100)            0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_12 (Dense)             (None, 106)               21306     
Total params: 202,206
Trainable params: 202,206
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
2/2 [==============================] - 0s - loss: 4.6915 - acc: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s - loss: 4.4362 - acc: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s - loss: 4.1477 - acc: 1.0000
Epoch 4/100

2/2 [==============================] - 0s - loss: 2.0441e-04 - acc: 1.0000
Epoch 86/100
2/2 [==============================] - 0s - loss: 2.0405e-04 - acc: 1.0000
Epoch 87/100
2/2 [==============================] - 0s - loss: 2.0345e-04 - acc: 1.0000
Epoch 88/100
2/2 [==============================] - 0s - loss: 2.0291e-04 - acc: 1.0000
Epoch 89/100
2/2 [==============================] - 0s - loss: 2.0244e-04 - acc: 1.0000
Epoch 90/100
2/2 [==============================] - 0s - loss: 2.0178e-04 - acc: 1.0000
Epoch 91/100
2/2 [==============================] - 0s - loss: 2.0131e-04 - acc: 1.0000
Epoch 92/100
2/2 [==============================] - 0s - loss: 2.0083e-04 - acc: 1.0000
Epoch 93/100
2/2 [==============================] - 0s - loss: 2.0005e-04 - acc: 1.0000
Epoch 94/100
2/2 [==============================] - 0s - loss: 1.9987e-04 - acc: 1.0000
Epoch 95/100
2/2 [==============================] - 0s - loss: 1.9919e-04 - acc: 1.0000
Epoch 96/100
2/2 [===========================

In [ ]:
model.

In [23]:
model.predict(np.array([x[1]]))

array([[[[  5.38322965e-05,   6.13281518e-05,   9.56183339e-06,
            3.51313465e-05,   4.28525091e-05,   2.60372835e-05,
            3.78655277e-05,   1.57424874e-05,   2.34715644e-05,
            2.62764534e-05,   8.81645428e-06,   5.55983497e-05,
            1.87348942e-05,   9.97261524e-01,   9.58101100e-06,
            3.08546150e-05,   2.73630067e-05,   7.40136893e-05,
            3.00108568e-05,   1.40599850e-05,   1.14558743e-05,
            4.55732079e-05,   1.93615579e-05,   9.55293672e-06,
            3.71718343e-05,   1.18684529e-05,   2.69043030e-05,
            1.44839287e-05,   1.31775487e-05,   2.32182083e-05,
            2.59079861e-05,   1.38583300e-05,   3.03715369e-05,
            4.00349927e-05,   1.49775988e-05,   1.72680211e-05,
            7.83308678e-06,   3.65528467e-05,   2.09940954e-05,
            1.27572048e-05,   4.50438965e-05,   2.90151529e-05,
            1.47320961e-05,   2.06013701e-05,   1.43498783e-05,
            2.67226460e-05,   3.21216576

In [35]:
Bidirectional()

TypeError: __init__() missing 1 required positional argument: 'layer'

In [82]:
x.shape

(106000,)

In [83]:
x = np.random.rand(100*106*10)

In [94]:
x = x.reshape(106, 1, 100, 10, 1)

In [95]:
y = np.arange(106)

In [96]:
y = y.reshape(106, 1)

In [97]:
y = to_categorical(y)[:10]

In [101]:
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(2, 2), input_shape=(None, 106, 100, 1)))
seq.add(Dense(y_start.shape[1], activation="softmax"))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')
seq.fit(x, y)

ValueError: Error when checking input: expected conv_lst_m2d_16_input to have shape (None, None, 106, 100, 1) but got array with shape (106, 1, 100, 10, 1)